In [16]:
import pandas as pd
import re
import requests

#イギリスの記事を取得
df = pd.read_json('jawiki-country.json.gz',lines=True)
uk = df.query('title == "イギリス"')['text'].values[0]

#ukの中身を1行ずつ配列に格納
uk_list = uk.split("\n")

#uk_listから1行ずつ取り出してカテゴリー名を宣言している確認
#re.searchで、"|"から始まり、" ="が間にあるような文字列を取得
    #例) |略名  =イギリス
#re.subで(\s.*=.+ or \|) を削除してkeyを取得し、valueは('\|.* =' or ' or [+ or ]+)を削除して取得する
dic = {}
for column in uk_list:
    if re.search(r'\|(.+?)\s=\s*(.+)', column):
        temp = re.search(r'\|(.+?)\s=\s*(.+)',column).group()
        dic[re.sub(r'\s.*=.+|\|','',temp)] = re.sub(r'\|.*\s=|\'|\[+|\]|<.*>','',temp)
        
#--------------------------------------------------------ここまではknock28と一緒--------------------------------------------------------------

#webAPIは、ウェブサービスが他のプログラムに対して自分の機能やデータを提供するために使われる
#Session()がなくてもアクセスできるが、使わないと都度ログイン処理が必要になり、サーバに負荷がかかったり、パフォーマンスが低下したりする
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"
 
PARAMS = {
    "action": "query",
    "format": "json",
    "prop": "imageinfo",
    "titles": f"File:{dic['国旗画像']}",
    "iiprop":"url"
}

#getメソッドでAPIからデータを取得、返り値はレスポンスオブジェクト、URL属性でアクセスするURLを指定、paramsでアクセスしたいURLにパラメータを付与できる(?より後がパラメータ)
R = S.get(url=URL, params=PARAMS)
#print(R.url),                  https://en.wikipedia.org/w/api.php?action=query&format=json&prop=imageinfo&titles=File%3A+Flag+of+the+United+Kingdom.svg&iiprop=url　(PAGESの情報が記載されている)
#paramsを引数として渡さないとき, https://en.wikipedia.org/w/api.php

#jsonメソッドでpythonの辞書オブジェクトに変換
DATA = R.json()

PAGES = DATA["query"]["pages"]
# print(PAGES)
# {'23473560': {'pageid': 23473560, 'ns': 6, 'title': 'File:Flag of the United Kingdom.svg', 'imagerepository': 'local', 'imageinfo': [{'url': 'https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg', 'descriptionurl': 'https://en.wikipedia.org/wiki/File:Flag_of_the_United_Kingdom.svg', 'descriptionshorturl': 'https://en.wikipedia.org/w/index.php?curid=23473560'}]}}

for k, v in PAGES.items():
    #print(v)、辞書型の入れ子になっている
    print(v["imageinfo"][0]["url"])

https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg
